In [1]:
!pip install .

Processing /Users/poramat/Desktop/qwanta
  Preparing metadata (setup.py) ... done
  Created wheel for qwanta: filename=qwanta-0.0.3-py3-none-any.whl size=40226 sha256=e6b73bc0b4af960c54f86a9fb4146e2b619ff75d892b546f2fc9c7dd72b6c93e
  Stored in directory: /private/var/folders/b7/kcj1zfy15d35hgd16vscfs0r0000gn/T/pip-ephem-wheel-cache-j81clzwm/wheels/0b/13/89/5198641a501d526f23394513838ddbdf7f2231a71be219fbb1
Successfully built qwanta
  Attempting uninstall: qwanta
    Found existing installation: qwanta 0.0.3
    Uninstalling qwanta-0.0.3:
      Successfully uninstalled qwanta-0.0.3


In [2]:
import numpy as np
from qwanta import Xperiment, Configuration
import matplotlib.pyplot as plt
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import dill
import requests
import ray

In [3]:
loss = 0.1 # dB/km # Photon loss
p_dep = [0.97, 0.01, 0.01, 0.01] # Depolarizing Channel 
gateErr = 0.01 # Gate error -> [Do, Mix] 
measurement_error = 0.01 # Measurement error
# Memory error
def memError(time, tau=10**(-3)):
    p = 0.75*np.e**(-1*(time/tau)) + 0.25 
    return [p, (1- p)/3, (1- p)/3, (1- p)/3]

In [4]:
nodes_info_exp = {
            'A': {'coordinate': (0, 0, 0)},
            'B': {'coordinate': (100, 0, 0)},
            'C': {'coordinate': (200, 0, 0)},
            'numPhysicalBuffer': 20,
            'numInternalEncodingBuffer': 20,
            'numInternalDetectingBuffer': 10,
            'numInternalInterfaceBuffer': 2,
        }


Quantum_topology = {
            ('A', 'B'): {
            'connection-type': 'Space',
            'function': p_dep,
            'loss': loss,
            'light speed': 300000, # km/s
            'Pulse rate': 0.0001, # waiting time for next qubit (interval)
            'memory function': 1,
            'depolarlizing error': [1, 0, 0, 0],
            },
            ('B', 'C'): {
            'connection-type': 'Space',
            'function': p_dep,
            'loss': loss,
            'light speed': 300000,
            'Pulse rate': 0.0001,
            'memory function': 1,
            'depolarlizing error': [1, 0, 0, 0],
            },
            ('A', 'C'): {
            'connection-type': 'Space',
            'function': p_dep,
            'loss': loss,
            'light speed': 300000,
            'Pulse rate': 0.0001,
            'memory function': 1,
            'depolarlizing error': [1, 0, 0, 0],
            },
        }


In [5]:
expers = Xperiment(
timelines_path = 'exper_id3_selectedStats_2hops.xlsx', 
nodes_info_exp = nodes_info_exp,
edges_info_exp = Quantum_topology,
gate_error = gateErr,
measurement_error = measurement_error,
memory_time = memError
)

In [6]:
expers.timelines

{'Sheet1': [{'Main Process': 'Generate physical Bell pair',
   'Edges': ['A', 'B'],
   'Label in': 'Physical qubit',
   'Label out': 'physical Bell pair',
   'Protocol': 'standard',
   'Resource Type': 'Physical',
   'Num Trials': True,
   'Note': nan,
   'isSuccess': 0},
  {'Main Process': 'Generate physical Bell pair',
   'Edges': ['B', 'C'],
   'Label in': 'Physical qubit',
   'Label out': 'physical Bell pair',
   'Protocol': 'standard',
   'Resource Type': 'Physical',
   'Num Trials': True,
   'Note': nan,
   'isSuccess': 0},
  {'Main Process': 'Entanglement swapping',
   'Edges': ['A', 'B', 'C'],
   'Label in': 'physical Bell pair',
   'Label out': 'Swapped',
   'Protocol': 'standard',
   'Resource Type': 'Physical',
   'Num Trials': True,
   'Note': nan,
   'isSuccess': 0},
  {'Main Process': 'Fidelity estimation',
   'Edges': ['A', 'C'],
   'Label in': 'Swapped',
   'Label out': 'Physical qubit',
   'Protocol': 'standard',
   'Resource Type': 'Physical',
   'Num Trials': 9000,
 

In [7]:
expers.validate(show_message=True, vis=True)


Validating experiment: Sheet1
---------------------------------------------
Limited Process: 
1. Fidelity estimation : End-to-end process -> ['A', 'C']
Topology provied is connected graph...
Not all edge have Generate physical Bell pair process, the missing edges are, 
('A', 'B')
('A', 'C')
('A', 'C')
('B', 'C')
Limited process: Fidelity estimation
('A', 'C') is reachable to fundamental resource processes...
[Sheet1] all status checked 
All timeline and topology of all experiments are validated, you are good to execute Experiment.run() command!                 
Another error that is not currently check is the number of qubits needed to completed the task.


True

In [8]:
expers.execute(save_result=True)

{'Sheet1': {'numPhysicalBuffer': 20,
  'numInternalEncodingBuffer': 20,
  'numInternalDetectingBuffer': 10,
  'numInternalInterfaceBuffer': 2,
  'memory_function': <function __main__.memError(time, tau=0.001)>,
  'gate_error': 0.01,
  'measurement_error': 0.01,
  'timeline': [{'Main Process': 'Generate physical Bell pair',
    'Edges': ['A', 'B'],
    'Label in': 'Physical qubit',
    'Label out': 'physical Bell pair',
    'Protocol': 'standard',
    'Resource Type': 'Physical',
    'Num Trials': True,
    'Note': nan,
    'isSuccess': 0},
   {'Main Process': 'Generate physical Bell pair',
    'Edges': ['B', 'C'],
    'Label in': 'Physical qubit',
    'Label out': 'physical Bell pair',
    'Protocol': 'standard',
    'Resource Type': 'Physical',
    'Num Trials': True,
    'Note': nan,
    'isSuccess': 0},
   {'Main Process': 'Entanglement swapping',
    'Edges': ['A', 'B', 'C'],
    'Label in': 'physical Bell pair',
    'Label out': 'Swapped',
    'Protocol': 'standard',
    'Resource

In [9]:
object_methods = [method_name for method_name in dir(expers) if callable(getattr(expers, method_name))]
object_methods

['__class__',
 '__delattr__',
 '__dir__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'execute',
 'validate']

In [10]:
import random

In [17]:
random.choices(['I', 'X', 'Z', 'Y'], weights=[0.25, 0.25, 0.25, 0.25])

['I']